In [70]:
# Dependencies


import numpy as np
import functools as ft
import networkx as nx
from sklearn.base import BaseEstimator, ClassifierMixin
from datasets import load_dataset
from sklearn.model_selection import cross_val_score, ShuffleSplit

In [2]:
# Basic arithemitic operations


def hdv(d):
    return np.random.choice([-1, 1], d)


def bind(xs):
    return ft.reduce(lambda x, y: x * y, xs)


def bundle(xs):
    return ft.reduce(lambda x, y: x + y, xs)


def similarity(A, B):
    return np.dot(A, B) / len(A)


def cosine_similarity(A, B):
    dot_product = np.dot(A, B)
    norm_A = np.linalg.norm(A)
    norm_B = np.linalg.norm(B)

    if norm_A == 0 or norm_B == 0:
        return 0

    return dot_product / (norm_A * norm_B)

In [3]:
# Memory


class ItemMemory:
    def __init__(self, vectors=[]):
        self.vectors = vectors

    def addVector(self, label, V):
        self.vectors.append((label, V))

    def cleanup(self, V):
        return max(self.vectors, key=lambda x: cosine_similarity(V, x[1]))

In [4]:
# Stochastic arithemtic operations


def weightedAverage(A, B, p, q):
    return np.fromiter(
        map(lambda t: np.random.choice([t[0], t[1]], p=[p, q]), zip(A, B)),
        dtype=np.int_,
    )


def hdvA(B, a):
    return weightedAverage(B, -B, (a + 1) / 2, (1 - a) / 2)


def hdvW(B, w):
    start = round(w * len(B))
    head = B[:start]
    tail = B[start:] * -1
    return np.concatenate([head, tail])

In [5]:
# # Basic tests

# A, B = hdv(10000), hdv(10000)
# C = weightedAverage(A, B, 0.90, 0.1)
# # print(A)
# # print(B)
# # print(C)
# # print(cosine_similarity(A, B))
# # print(cosine_similarity(B, C))

# a = 0.75

# Ba = hdvA(B, a)
# Bb = hdvA(B, 0.6)
# print(cosine_similarity(B, Ba))
# # print(cosine_similarity(Ba, Bb))
# print((cosine_similarity(B, Ba) + 1) / 2)
# print((similarity(B, Ba) + 1) / 2)

# Bw = hdvW(B, 0.1)
# print(B)
# print(Bw)
# print(cosine_similarity(B, Bw))
# print((cosine_similarity(B, Bw) + 1) / 2)

# Bw1 = hdvW(B, 0.0248)
# Bw2 = hdvW(B, 0.015)
# print(cosine_similarity(Bw1, Bw2))

In [6]:
# transform(graphs, 0.45, 5)[0].nodes

In [87]:
def processDataset(dataset):
    graphs = []
    labels = []

    for graph in dataset:
        G = nx.Graph()
        G.add_edges_from(zip(graph["edge_index"][0], graph["edge_index"][1]))
        graphs.append(G)
        labels.append(graph["y"][0])

    return (graphs, labels)


def transform(X, alpha, digits):
    graphs = []
    for graph in X:
        gpr = nx.pagerank(graph, alpha)
        nodes = dict()
        for key, value in gpr.items():
            nodes[key] = str(round(value, digits))
        H = nx.relabel_nodes(graph, nodes)
        graphs.append(H)
    return graphs


def encodeGraph(graph, vertices, dimensions, base):
    for node in graph.nodes:
        if node not in vertices:
            vertices[node] = hdvW(base, float(node))

    Edges = []

    for edge in graph.edges:
        v1 = vertices[edge[0]]
        v2 = vertices[edge[1]]
        E = bind([v1, v2])
        Edges.append(E)

    Graph = bundle(Edges)

    return Graph

In [81]:
class GraphHD(BaseEstimator, ClassifierMixin):
    def __init__(self, alpha=0.45, digits=4, dimensions=10000, step=20):
        self.alpha = alpha
        self.digits = digits
        self.dimensions = dimensions
        self.step = step
        self.B = hdv(dimensions)
        self.memory = ItemMemory([])

    def fit(self, X, y):
        self.vertices = dict()
        self.labels = list(set(y))
        dictLabels = dict()

        graphs = transform(X, self.alpha, self.digits)

        for label in self.labels:
            dictLabels[label] = []

        for i in range(len(graphs)):
            Graph = encodeGraph(graphs[i], self.vertices, self.dimensions, self.B)
            dictLabels[y[i]].append(Graph)

        for key, value in dictLabels.items():
            for i in range(0, len(value), self.step):
                H = bundle(value[i : i + self.step])
                self.memory.addVector(str(key), H)

        return self

    def predict(self, X):
        p = []
        s = []

        graphs = transform(X, self.alpha, self.digits)

        for testGraph in graphs:
            queryVector = encodeGraph(testGraph, self.vertices, self.dimensions, self.B)
            cleanVector = self.memory.cleanup(queryVector)

            p.append(int(cleanVector[0]))
            # s.append(cosine_similarity(queryVector, cleanVector[1]))

        # print("%.5f" % round(np.mean(s), 5), "0:", p.count(0), "1:", p.count(1))
        return p

In [80]:
MUTAG = load_dataset("graphs-datasets/MUTAG")["train"]
PROTEINS = load_dataset("graphs-datasets/PROTEINS")["train"]
AIDS = load_dataset("graphs-datasets/AIDS")["full"]
IMDB = load_dataset("graphs-datasets/IMDB-BINARY")["train"]

In [57]:
(graphs, labels) = processDataset(MUTAG)

In [86]:
FOLDS, REPS, CV = 10, 3, FOLDS
ALPHA, DIGITS, DIMENSIONS, STEP = 0.85, 5, 10000, 20


def main():
    clf = GraphHD(ALPHA, DIGITS, DIMENSIONS, STEP)
    sum = 0
    for i in range(REPS):
        # CV = ShuffleSplit(n_splits=5, test_size=0.25)  # random_state=0
        scores = cross_val_score(clf, graphs, labels, n_jobs=-1, cv=CV, verbose=0)
        sum += scores.mean()
        print(i, "->", "%.5f" % scores.mean())
        del scores

    print("S => %.5f" % (sum / REPS))


main()

0 -> 0.85029
1 -> 0.85029
2 -> 0.85029
S => 0.85029
